# advent of code 2024 - [dayX](https://adventofcode.com/2024/day/X)

In [ ]:
import os

NEO4J_URI = os.environ['NEO4J_URI']
NEO4J_USERNAME = os.environ['NEO4J_USERNAME']
NEO4J_PASSWORD = os.environ['NEO4J_PASSWORD']


In [ ]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

from neo4j import GraphDatabase
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

import pandas as pd

In [ ]:
def gen_lists(file='input.txt'):
    """Generates tuples of integers"""
    file = open(file, 'r')
    for ix, line in enumerate(file):
        for jx, c in enumerate(line):
            yield ix, jx, c

## Neo4j-based solution

### Parsing

In [ ]:
tiles = [{'row':ix, 'col':jx, 'val':c} for ix, jx, c in list(gen_lists())]

### Ingestion

In [ ]:
query_ingest = """
UNWIND $tiles AS tile
CREATE (:Tile {row:tile.row, col:tile.col, val:tile.val} )
"""

gds.run_cypher(query_ingest, {"tiles":tiles})

### Building lattice

In [ ]:
gds.run_cypher('CREATE INDEX tile_row IF NOT EXISTS FOR (r:Tile) ON (r.row)')
gds.run_cypher('CREATE INDEX tile_col IF NOT EXISTS FOR (r:Tile) ON (r.col)')
gds.run_cypher('CREATE INDEX tile_val IF NOT EXISTS FOR (r:Tile) ON (r.val)')

In [ ]:
gds.run_cypher("""
MATCH (t:Tile)
WITH t.row AS row_num, t ORDER BY t.col
WITH row_num, collect(t) AS row
CALL apoc.nodes.link(row, 'EAST')
""")

In [ ]:
gds.run_cypher("""
MATCH (t:Tile)
WITH t.col AS col_num, t ORDER BY t.row
WITH col_num, collect(t) AS col
CALL apoc.nodes.link(col, 'SOUTH')
""")

In [ ]:
gds.run_cypher("""
MATCH (a)-[:EAST]->(b)-[:SOUTH]->(c)<-[:EAST]->(d)<-[:SOUTH]-(a)
MERGE (a)-[:SOUTH_EAST]->(c)
MERGE (b)-[:SOUTH_WEST]->(d)
""")

### Part 1

In [ ]:
gds.run_cypher("""
CALL (){
MATCH path=(a {val:'X'})-[r1]->(b {val:'M'})-[r2]->(c {val:'A'})-[r3]->(d {val:'S'})
WHERE type(r1) = type(r2) = type(r3)
RETURN path

UNION

MATCH path=(a {val:'S'})-[r1]->(b {val:'A'})-[r2]->(c {val:'M'})-[r3]->(d {val:'X'})
WHERE type(r1) = type(r2) = type(r3)
RETURN path
}
RETURN count(path) AS part1
""")

### Part 2

In [ ]:
gds.run_cypher("""
MATCH path1=(a {val:'M'})-[:SOUTH_EAST]-(b {val:'A'})-[:SOUTH_EAST]-(c {val:'S'}),
path2 = (d {val:'M'})-[:SOUTH_WEST]-(b)-[:SOUTH_WEST]-(e {val:'S'})
RETURN count(*) AS part2
""")